In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
import datetime
from collections import namedtuple
from typing import Any, Dict, List, Union


import fitdecode

In [59]:
FitReferenceField = namedtuple(
    "FitReferenceField", fitdecode.types.ReferenceField.__slots__
)
FitFieldDefinition = namedtuple(
    "FitFieldDefinition", fitdecode.types.FieldDefinition.__slots__
)
FitBaseType = namedtuple("FitBaseType", fitdecode.types.BaseType.__slots__)
FitFieldType = namedtuple("FitFieldType", fitdecode.types.FieldType.__slots__)
FitField = namedtuple("FitField", fitdecode.types.Field.__slots__)
FitMessageType = namedtuple("FitMessageType", fitdecode.types.MessageType.__slots__)
FitSubField = namedtuple("FitSubField", fitdecode.types.SubField.__slots__)
FitFieldData = namedtuple("FitFieldData", fitdecode.types.FieldData.__slots__)

FitComponentField = namedtuple(
    "FitComponentField", fitdecode.types.ComponentField.__slots__
)


FieldTypePyType = Union[fitdecode.types.FieldType, fitdecode.types.BaseType]


def parse_referene_field(record: fitdecode.types.ReferenceField) -> FitReferenceField:
    return FitReferenceField(
        record.name, record.def_num, record.value, record.raw_value
    )


def parse_compnent_field(record: fitdecode.types.ComponentField) -> FitComponentField:
    return FitComponentField(
        record.name,
        record.def_num,
        record.scale,
        record.offset,
        record.units,
        record.accumulate,
        record.bits,
        record.bit_offset,
    )


def parse_subfield(record: fitdecode.types.SubField) -> FitSubField:
    return FitSubField(
        record.name,
        record.def_num,
        parse_field_type(record.type),
        record.scale,
        record.offset,
        record.units,
        [parse_compnent_field(field) for field in record.components]
        if record.components is not None
        else None,
        [parse_referene_field(referene_field) for referene_field in record.ref_fields]
        if record.ref_fields is not None
        else None,
    )


def parse_field_definition(
    record: fitdecode.types.FieldDefinition,
) -> FitFieldDefinition:
    return FitFieldDefinition(
        parse_field(record.field),
        record.def_num,
        parse_field_type(record.base_type),
        record.size,
    )


def parse_base_type(record: fitdecode.types.BaseType) -> FitBaseType:
    return FitBaseType(
        record.name, record.identifier, record.fmt, record.size, "record.parse"
    )


def parse_field_type(record: FieldTypePyType) -> FieldTypePyType:
    if isinstance(record, fitdecode.types.FieldType):
        return FitFieldType(
            record.name, parse_base_type(record.base_type), "record.enum"
        )
    if isinstance(record, fitdecode.types.BaseType):
        return parse_base_type(record)


def parse_field(record: fitdecode.types.Field) -> FitField:

    return FitField(
        record.name,
        "parse_field_type(record.type)",
        record.def_num,
        record.scale,
        record.offset,
        record.units,
        [parse_compnent_field(field) for field in record.components]
        if record.components is not None
        else None,
        [parse_subfield(subfield) for subfield in record.subfields]
        if record.subfields is not None
        else None,
    )


def parse_message_type(record: fitdecode.types.MessageType) -> FitMessageType:
    fields = {}
    for field in record.fields:
        fields[field] = parse_field(record.fields[field])
    return FitMessageType(record.name, record.mesg_num, fields)


def parse_field_data(record: fitdecode.types.FieldData) -> FitFieldData:
    return FitFieldData(
        "parse_field_definition(record.field_def)",
        parse_field(record.field),
        record.parent_field,
        record.value,
        record.raw_value,
        record.units,
    )

In [60]:
FitHeaderRow = namedtuple("FitHeaderRow", fitdecode.records.FitHeader.__slots__)

FitDefinitionRow = namedtuple(
    "FitDefinitionRow", fitdecode.records.FitDefinitionMessage.__slots__
)

FitDataMessageRecord = namedtuple(
    "FitDataMessageRecord", fitdecode.records.FitDataMessage.__slots__
)


def parse_header_record(record: fitdecode.records.FitHeader) -> FitHeaderRow:
    return FitHeaderRow(
        record.header_size,
        record.proto_ver,
        record.profile_ver,
        record.body_size,
        record.crc,
        record.crc_matched,
        record.chunk,
    )


def parse_definition_record(
    record: fitdecode.records.FitDefinitionMessage,
) -> FitHeaderRow:
    return FitDefinitionRow(
        record.is_developer_data,
        record.local_mesg_num,
        record.time_offset,
        "parse_message_type(record.mesg_type)",
        record.global_mesg_num,
        record.endian,
        [parse_field_definition(field_def) for field_def in record.field_defs]
        if record.field_defs is not None
        else None,
        record.dev_field_defs,
        record.chunk,
    )


def parse_data_message_record(
    record: fitdecode.records.FitDataMessage,
) -> FitDataMessageRecord:
    return FitDataMessageRecord(
        record.is_developer_data,
        record.local_mesg_num,
        record.time_offset,
        "parse_definition_record(record.def_mesg)",
        [parse_field_data(field) for field in record.fields]
        if record.fields is not None
        else None,
        record.chunk,
    )

In [61]:
# with fitdecode.FitReader("fit_files/Afternoon_Ride.fit") as fit:
#     fit_frames = [frame for frame in fit]

with fitdecode.FitReader("fit_files/The_Sufferfest_G_O_A_T.fit") as fit:
    fit_frames = [frame for frame in fit]

fit_frames[:15]

In [62]:
fit_header = parse_header_record(fit_frames[0])
fit_header

FitHeaderRow(header_size=14, proto_ver=(2, 0), profile_ver=(20, 42), body_size=40290, crc=28244, crc_matched=True, chunk=None)

In [63]:
fit_definition_message = parse_definition_record(fit_frames[1])
fit_definition_message

FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type='parse_message_type(record.mesg_type)', global_mesg_num=0, endian='<', field_defs=[FitFieldDefinition(field=FitField(name='type', type='parse_field_type(record.type)', def_num=0, scale=None, offset=None, units=None, components=None, subfields=None), def_num=0, base_type=FitBaseType(name='enum', identifier=0, fmt='B', size=1, parse='record.parse'), size=1), FitFieldDefinition(field=FitField(name='manufacturer', type='parse_field_type(record.type)', def_num=1, scale=None, offset=None, units=None, components=None, subfields=None), def_num=1, base_type=FitBaseType(name='uint16', identifier=132, fmt='H', size=2, parse='record.parse'), size=2), FitFieldDefinition(field=FitField(name='product', type='parse_field_type(record.type)', def_num=2, scale=None, offset=None, units=None, components=None, subfields=[FitSubField(name='favero_product', def_num=2, type=FitFieldType(name='favero_product', base_type=FitB

In [64]:
FitDefinitionRow(
    is_developer_data=False,
    local_mesg_num=0,
    time_offset=None,
    mesg_type="parse_message_type(record.mesg_type)",
    global_mesg_num=0,
    endian="<",
    field_defs=[
        FitFieldDefinition(
            field=FitField(
                name="type",
                type="parse_field_type(record.type)",
                def_num=0,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            def_num=0,
            base_type=FitBaseType(
                name="enum", identifier=0, fmt="B", size=1, parse="record.parse"
            ),
            size=1,
        ),
        FitFieldDefinition(
            field=FitField(
                name="manufacturer",
                type="parse_field_type(record.type)",
                def_num=1,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            def_num=1,
            base_type=FitBaseType(
                name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
            ),
            size=2,
        ),
        FitFieldDefinition(
            field=FitField(
                name="product",
                type="parse_field_type(record.type)",
                def_num=2,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            def_num=2,
            base_type=FitBaseType(
                name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
            ),
            size=2,
        ),
        FitFieldDefinition(
            field=FitField(
                name="serial_number",
                type="parse_field_type(record.type)",
                def_num=3,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            def_num=3,
            base_type=FitBaseType(
                name="uint32z", identifier=140, fmt="I", size=4, parse="record.parse"
            ),
            size=4,
        ),
        FitFieldDefinition(
            field=FitField(
                name="time_created",
                type="parse_field_type(record.type)",
                def_num=4,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            def_num=4,
            base_type=FitBaseType(
                name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
            ),
            size=4,
        ),
    ],
    dev_field_defs=[],
    chunk=None,
)

FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type='parse_message_type(record.mesg_type)', global_mesg_num=0, endian='<', field_defs=[FitFieldDefinition(field=FitField(name='type', type='parse_field_type(record.type)', def_num=0, scale=None, offset=None, units=None, components=None, subfields='subfields'), def_num=0, base_type=FitBaseType(name='enum', identifier=0, fmt='B', size=1, parse='record.parse'), size=1), FitFieldDefinition(field=FitField(name='manufacturer', type='parse_field_type(record.type)', def_num=1, scale=None, offset=None, units=None, components=None, subfields='subfields'), def_num=1, base_type=FitBaseType(name='uint16', identifier=132, fmt='H', size=2, parse='record.parse'), size=2), FitFieldDefinition(field=FitField(name='product', type='parse_field_type(record.type)', def_num=2, scale=None, offset=None, units=None, components=None, subfields='subfields'), def_num=2, base_type=FitBaseType(name='uint16', identifier=132, fmt='H', s

In [65]:
parse_data_message_record(fit_frames[2])

FitDataMessageRecord(is_developer_data=False, local_mesg_num=0, time_offset=None, def_mesg='parse_definition_record(record.def_mesg)', fields=[FitFieldData(field_def='parse_field_definition(record.field_def)', field=FitField(name='type', type='parse_field_type(record.type)', def_num=0, scale=None, offset=None, units=None, components=None, subfields=None), parent_field=None, value='activity', raw_value=4, units=None), FitFieldData(field_def='parse_field_definition(record.field_def)', field=FitField(name='manufacturer', type='parse_field_type(record.type)', def_num=1, scale=None, offset=None, units=None, components=None, subfields=None), parent_field=None, value='the_sufferfest', raw_value=282, units=None), FitFieldData(field_def='parse_field_definition(record.field_def)', field=FitField(name='product', type='parse_field_type(record.type)', def_num=2, scale=None, offset=None, units=None, components=None, subfields=[FitSubField(name='favero_product', def_num=2, type=FitFieldType(name='fav

In [66]:
FitDataMessageRecord(
    is_developer_data=False,
    local_mesg_num=0,
    time_offset=None,
    def_mesg="parse_definition_record(record.def_mesg)",
    fields=[
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="type",
                type="parse_field_type(record.type)",
                def_num=0,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            parent_field=None,
            value="activity",
            raw_value=4,
            units=None,
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="manufacturer",
                type="parse_field_type(record.type)",
                def_num=1,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            parent_field=None,
            value="the_sufferfest",
            raw_value=282,
            units=None,
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="product",
                type="parse_field_type(record.type)",
                def_num=2,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            parent_field=None,
            value=1231,
            raw_value=1231,
            units=None,
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="serial_number",
                type="parse_field_type(record.type)",
                def_num=3,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            parent_field=None,
            value=12345,
            raw_value=12345,
            units=None,
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="time_created",
                type="parse_field_type(record.type)",
                def_num=4,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields="subfields",
            ),
            parent_field=None,
            value=datetime.datetime(
                2020, 5, 22, 16, 16, 16, tzinfo=datetime.timezone.utc
            ),
            raw_value=959098576,
            units=None,
        ),
    ],
    chunk=None,
)

FitDataMessageRecord(is_developer_data=False, local_mesg_num=0, time_offset=None, def_mesg='parse_definition_record(record.def_mesg)', fields=[FitFieldData(field_def='parse_field_definition(record.field_def)', field=FitField(name='type', type='parse_field_type(record.type)', def_num=0, scale=None, offset=None, units=None, components=None, subfields='subfields'), parent_field=None, value='activity', raw_value=4, units=None), FitFieldData(field_def='parse_field_definition(record.field_def)', field=FitField(name='manufacturer', type='parse_field_type(record.type)', def_num=1, scale=None, offset=None, units=None, components=None, subfields='subfields'), parent_field=None, value='the_sufferfest', raw_value=282, units=None), FitFieldData(field_def='parse_field_definition(record.field_def)', field=FitField(name='product', type='parse_field_type(record.type)', def_num=2, scale=None, offset=None, units=None, components=None, subfields='subfields'), parent_field=None, value=1231, raw_value=1231,

In [67]:
parse_definition_record(fit_frames[3]), parse_data_message_record(fit_frames[4])

(FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type='parse_message_type(record.mesg_type)', global_mesg_num=23, endian='<', field_defs=[FitFieldDefinition(field=FitField(name='timestamp', type='parse_field_type(record.type)', def_num=253, scale=None, offset=None, units='s', components=None, subfields=None), def_num=253, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', size=4, parse='record.parse'), size=4), FitFieldDefinition(field=FitField(name='battery_status', type='parse_field_type(record.type)', def_num=11, scale=None, offset=None, units=None, components=None, subfields=None), def_num=11, base_type=FitBaseType(name='uint8', identifier=2, fmt='B', size=1, parse='record.parse'), size=1)], dev_field_defs=[], chunk=None),
 FitDataMessageRecord(is_developer_data=False, local_mesg_num=0, time_offset=None, def_mesg='parse_definition_record(record.def_mesg)', fields=[FitFieldData(field_def='parse_field_definition(record.field_def)', field=

In [68]:
(
    FitDefinitionRow(
        is_developer_data=False,
        local_mesg_num=0,
        time_offset=None,
        mesg_type="parse_message_type(record.mesg_type)",
        global_mesg_num=23,
        endian="<",
        field_defs=[
            FitFieldDefinition(
                field=FitField(
                    name="timestamp",
                    type="parse_field_type(record.type)",
                    def_num=253,
                    scale=None,
                    offset=None,
                    units="s",
                    components=None,
                    subfields="subfields",
                ),
                def_num=253,
                base_type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                size=4,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="battery_status",
                    type="parse_field_type(record.type)",
                    def_num=11,
                    scale=None,
                    offset=None,
                    units=None,
                    components=None,
                    subfields="subfields",
                ),
                def_num=11,
                base_type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                size=1,
            ),
        ],
        dev_field_defs=[],
        chunk=None,
    ),
    FitDataMessageRecord(
        is_developer_data=False,
        local_mesg_num=0,
        time_offset=None,
        def_mesg="parse_definition_record(record.def_mesg)",
        fields=[
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="timestamp",
                    type="parse_field_type(record.type)",
                    def_num=253,
                    scale=None,
                    offset=None,
                    units="s",
                    components=None,
                    subfields="subfields",
                ),
                parent_field=None,
                value=datetime.datetime(
                    2020, 5, 22, 16, 58, 50, tzinfo=datetime.timezone.utc
                ),
                raw_value=959101130,
                units=None,
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="battery_status",
                    type="parse_field_type(record.type)",
                    def_num=11,
                    scale=None,
                    offset=None,
                    units=None,
                    components=None,
                    subfields="subfields",
                ),
                parent_field=None,
                value="good",
                raw_value=2,
                units=None,
            ),
        ],
        chunk=None,
    ),
)

(FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type='parse_message_type(record.mesg_type)', global_mesg_num=23, endian='<', field_defs=[FitFieldDefinition(field=FitField(name='timestamp', type='parse_field_type(record.type)', def_num=253, scale=None, offset=None, units='s', components=None, subfields='subfields'), def_num=253, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', size=4, parse='record.parse'), size=4), FitFieldDefinition(field=FitField(name='battery_status', type='parse_field_type(record.type)', def_num=11, scale=None, offset=None, units=None, components=None, subfields='subfields'), def_num=11, base_type=FitBaseType(name='uint8', identifier=2, fmt='B', size=1, parse='record.parse'), size=1)], dev_field_defs=[], chunk=None),
 FitDataMessageRecord(is_developer_data=False, local_mesg_num=0, time_offset=None, def_mesg='parse_definition_record(record.def_mesg)', fields=[FitFieldData(field_def='parse_field_definition(record.field

In [69]:
parse_definition_record(fit_frames[5]), parse_data_message_record(fit_frames[6])

(FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type='parse_message_type(record.mesg_type)', global_mesg_num=20, endian='<', field_defs=[FitFieldDefinition(field=FitField(name='timestamp', type='parse_field_type(record.type)', def_num=253, scale=None, offset=None, units='s', components=None, subfields=None), def_num=253, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', size=4, parse='record.parse'), size=4), FitFieldDefinition(field=FitField(name='power', type='parse_field_type(record.type)', def_num=7, scale=None, offset=None, units='watts', components=None, subfields=None), def_num=7, base_type=FitBaseType(name='uint16', identifier=132, fmt='H', size=2, parse='record.parse'), size=2), FitFieldDefinition(field=FitField(name='speed', type='parse_field_type(record.type)', def_num=6, scale=1000, offset=None, units='m/s', components=[FitComponentField(name='enhanced_speed', def_num=73, scale=1000, offset=None, units='m/s', accumulate=False,

In [72]:
(
    FitDefinitionRow(
        is_developer_data=False,
        local_mesg_num=0,
        time_offset=None,
        mesg_type="parse_message_type(record.mesg_type)",
        global_mesg_num=20,
        endian="<",
        field_defs=[
            FitFieldDefinition(
                field=FitField(
                    name="timestamp",
                    type="parse_field_type(record.type)",
                    def_num=253,
                    scale=None,
                    offset=None,
                    units="s",
                    components=None,
                    subfields=None,
                ),
                def_num=253,
                base_type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                size=4,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="power",
                    type="parse_field_type(record.type)",
                    def_num=7,
                    scale=None,
                    offset=None,
                    units="watts",
                    components=None,
                    subfields=None,
                ),
                def_num=7,
                base_type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                size=2,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="speed",
                    type="parse_field_type(record.type)",
                    def_num=6,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=[
                        FitComponentField(
                            name="enhanced_speed",
                            def_num=73,
                            scale=1000,
                            offset=None,
                            units="m/s",
                            accumulate=False,
                            bits=16,
                            bit_offset=0,
                        )
                    ],
                    subfields=None,
                ),
                def_num=6,
                base_type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                size=2,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="distance",
                    type="parse_field_type(record.type)",
                    def_num=5,
                    scale=100,
                    offset=None,
                    units="m",
                    components=None,
                    subfields=None,
                ),
                def_num=5,
                base_type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                size=4,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="cadence",
                    type="parse_field_type(record.type)",
                    def_num=4,
                    scale=None,
                    offset=None,
                    units="rpm",
                    components=None,
                    subfields=None,
                ),
                def_num=4,
                base_type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                size=1,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="heart_rate",
                    type="parse_field_type(record.type)",
                    def_num=3,
                    scale=None,
                    offset=None,
                    units="bpm",
                    components=None,
                    subfields=None,
                ),
                def_num=3,
                base_type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                size=1,
            ),
        ],
        dev_field_defs=[],
        chunk=None,
    ),
    FitDataMessageRecord(
        is_developer_data=False,
        local_mesg_num=0,
        time_offset=None,
        def_mesg="parse_definition_record(record.def_mesg)",
        fields=[
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="timestamp",
                    type="parse_field_type(record.type)",
                    def_num=253,
                    scale=None,
                    offset=None,
                    units="s",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=datetime.datetime(
                    2020, 5, 22, 16, 16, 16, tzinfo=datetime.timezone.utc
                ),
                raw_value=959098576,
                units=None,
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="power",
                    type="parse_field_type(record.type)",
                    def_num=7,
                    scale=None,
                    offset=None,
                    units="watts",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="watts",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="enhanced_speed",
                    type="parse_field_type(record.type)",
                    def_num=73,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0.0,
                units="m/s",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="speed",
                    type="parse_field_type(record.type)",
                    def_num=6,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=[
                        FitComponentField(
                            name="enhanced_speed",
                            def_num=73,
                            scale=1000,
                            offset=None,
                            units="m/s",
                            accumulate=False,
                            bits=16,
                            bit_offset=0,
                        )
                    ],
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0,
                units="m/s",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="distance",
                    type="parse_field_type(record.type)",
                    def_num=5,
                    scale=100,
                    offset=None,
                    units="m",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0,
                units="m",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="cadence",
                    type="parse_field_type(record.type)",
                    def_num=4,
                    scale=None,
                    offset=None,
                    units="rpm",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="rpm",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="heart_rate",
                    type="parse_field_type(record.type)",
                    def_num=3,
                    scale=None,
                    offset=None,
                    units="bpm",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="bpm",
            ),
        ],
        chunk=None,
    ),
)

(FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type='parse_message_type(record.mesg_type)', global_mesg_num=20, endian='<', field_defs=[FitFieldDefinition(field=FitField(name='timestamp', type='parse_field_type(record.type)', def_num=253, scale=None, offset=None, units='s', components=None, subfields=None), def_num=253, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', size=4, parse='record.parse'), size=4), FitFieldDefinition(field=FitField(name='power', type='parse_field_type(record.type)', def_num=7, scale=None, offset=None, units='watts', components=None, subfields=None), def_num=7, base_type=FitBaseType(name='uint16', identifier=132, fmt='H', size=2, parse='record.parse'), size=2), FitFieldDefinition(field=FitField(name='speed', type='parse_field_type(record.type)', def_num=6, scale=1000, offset=None, units='m/s', components=[FitComponentField(name='enhanced_speed', def_num=73, scale=1000, offset=None, units='m/s', accumulate=False,

In [73]:
parse_definition_record(fit_frames[7]), parse_data_message_record(fit_frames[8])

(FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type='parse_message_type(record.mesg_type)', global_mesg_num=19, endian='<', field_defs=[FitFieldDefinition(field=FitField(name='timestamp', type='parse_field_type(record.type)', def_num=253, scale=None, offset=None, units='s', components=None, subfields=None), def_num=253, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', size=4, parse='record.parse'), size=4), FitFieldDefinition(field=FitField(name='start_time', type='parse_field_type(record.type)', def_num=2, scale=None, offset=None, units=None, components=None, subfields=None), def_num=2, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', size=4, parse='record.parse'), size=4), FitFieldDefinition(field=FitField(name='total_elapsed_time', type='parse_field_type(record.type)', def_num=7, scale=1000, offset=None, units='s', components=None, subfields=None), def_num=7, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', 

In [74]:
(
    FitDefinitionRow(
        is_developer_data=False,
        local_mesg_num=0,
        time_offset=None,
        mesg_type="parse_message_type(record.mesg_type)",
        global_mesg_num=19,
        endian="<",
        field_defs=[
            FitFieldDefinition(
                field=FitField(
                    name="timestamp",
                    type="parse_field_type(record.type)",
                    def_num=253,
                    scale=None,
                    offset=None,
                    units="s",
                    components=None,
                    subfields=None,
                ),
                def_num=253,
                base_type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                size=4,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="start_time",
                    type="parse_field_type(record.type)",
                    def_num=2,
                    scale=None,
                    offset=None,
                    units=None,
                    components=None,
                    subfields=None,
                ),
                def_num=2,
                base_type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                size=4,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="total_elapsed_time",
                    type="parse_field_type(record.type)",
                    def_num=7,
                    scale=1000,
                    offset=None,
                    units="s",
                    components=None,
                    subfields=None,
                ),
                def_num=7,
                base_type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                size=4,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="total_distance",
                    type="parse_field_type(record.type)",
                    def_num=9,
                    scale=100,
                    offset=None,
                    units="m",
                    components=None,
                    subfields=None,
                ),
                def_num=9,
                base_type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                size=4,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="max_speed",
                    type="parse_field_type(record.type)",
                    def_num=14,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=[
                        FitComponentField(
                            name="enhanced_max_speed",
                            def_num=111,
                            scale=1000,
                            offset=None,
                            units="m/s",
                            accumulate=False,
                            bits=16,
                            bit_offset=0,
                        )
                    ],
                    subfields=None,
                ),
                def_num=14,
                base_type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                size=2,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="avg_speed",
                    type="parse_field_type(record.type)",
                    def_num=13,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=[
                        FitComponentField(
                            name="enhanced_avg_speed",
                            def_num=110,
                            scale=1000,
                            offset=None,
                            units="m/s",
                            accumulate=False,
                            bits=16,
                            bit_offset=0,
                        )
                    ],
                    subfields=None,
                ),
                def_num=13,
                base_type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                size=2,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="max_cadence",
                    type="parse_field_type(record.type)",
                    def_num=18,
                    scale=None,
                    offset=None,
                    units="rpm",
                    components=None,
                    subfields=[
                        FitSubField(
                            name="max_running_cadence",
                            def_num=18,
                            type=FitBaseType(
                                name="uint8",
                                identifier=2,
                                fmt="B",
                                size=1,
                                parse="record.parse",
                            ),
                            scale=None,
                            offset=None,
                            units="strides/min",
                            components=None,
                            ref_fields=[
                                FitReferenceField(
                                    name="sport",
                                    def_num=25,
                                    value="running",
                                    raw_value=1,
                                )
                            ],
                        )
                    ],
                ),
                def_num=18,
                base_type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                size=1,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="avg_cadence",
                    type="parse_field_type(record.type)",
                    def_num=17,
                    scale=None,
                    offset=None,
                    units="rpm",
                    components=None,
                    subfields=[
                        FitSubField(
                            name="avg_running_cadence",
                            def_num=17,
                            type=FitBaseType(
                                name="uint8",
                                identifier=2,
                                fmt="B",
                                size=1,
                                parse="record.parse",
                            ),
                            scale=None,
                            offset=None,
                            units="strides/min",
                            components=None,
                            ref_fields=[
                                FitReferenceField(
                                    name="sport",
                                    def_num=25,
                                    value="running",
                                    raw_value=1,
                                )
                            ],
                        )
                    ],
                ),
                def_num=17,
                base_type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                size=1,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="max_power",
                    type="parse_field_type(record.type)",
                    def_num=20,
                    scale=None,
                    offset=None,
                    units="watts",
                    components=None,
                    subfields=None,
                ),
                def_num=20,
                base_type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                size=2,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="avg_power",
                    type="parse_field_type(record.type)",
                    def_num=19,
                    scale=None,
                    offset=None,
                    units="watts",
                    components=None,
                    subfields=None,
                ),
                def_num=19,
                base_type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                size=2,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="max_heart_rate",
                    type="parse_field_type(record.type)",
                    def_num=16,
                    scale=None,
                    offset=None,
                    units="bpm",
                    components=None,
                    subfields=None,
                ),
                def_num=16,
                base_type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                size=1,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="avg_heart_rate",
                    type="parse_field_type(record.type)",
                    def_num=15,
                    scale=None,
                    offset=None,
                    units="bpm",
                    components=None,
                    subfields=None,
                ),
                def_num=15,
                base_type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                size=1,
            ),
            FitFieldDefinition(
                field=FitField(
                    name="total_work",
                    type="parse_field_type(record.type)",
                    def_num=41,
                    scale=None,
                    offset=None,
                    units="J",
                    components=None,
                    subfields=None,
                ),
                def_num=41,
                base_type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                size=4,
            ),
        ],
        dev_field_defs=[],
        chunk=None,
    ),
    FitDataMessageRecord(
        is_developer_data=False,
        local_mesg_num=0,
        time_offset=None,
        def_mesg="parse_definition_record(record.def_mesg)",
        fields=[
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="timestamp",
                    type="parse_field_type(record.type)",
                    def_num=253,
                    scale=None,
                    offset=None,
                    units="s",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=datetime.datetime(
                    2020, 5, 22, 16, 16, 16, tzinfo=datetime.timezone.utc
                ),
                raw_value=959098576,
                units=None,
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="start_time",
                    type="parse_field_type(record.type)",
                    def_num=2,
                    scale=None,
                    offset=None,
                    units=None,
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=datetime.datetime(
                    2020, 5, 22, 16, 16, 16, tzinfo=datetime.timezone.utc
                ),
                raw_value=959098576,
                units=None,
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="total_elapsed_time",
                    type="parse_field_type(record.type)",
                    def_num=7,
                    scale=1000,
                    offset=None,
                    units="s",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0,
                units="s",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="total_distance",
                    type="parse_field_type(record.type)",
                    def_num=9,
                    scale=100,
                    offset=None,
                    units="m",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0,
                units="m",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="enhanced_max_speed",
                    type="parse_field_type(record.type)",
                    def_num=111,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0.0,
                units="m/s",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="max_speed",
                    type="parse_field_type(record.type)",
                    def_num=14,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=[
                        FitComponentField(
                            name="enhanced_max_speed",
                            def_num=111,
                            scale=1000,
                            offset=None,
                            units="m/s",
                            accumulate=False,
                            bits=16,
                            bit_offset=0,
                        )
                    ],
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0,
                units="m/s",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="enhanced_avg_speed",
                    type="parse_field_type(record.type)",
                    def_num=110,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0.0,
                units="m/s",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="avg_speed",
                    type="parse_field_type(record.type)",
                    def_num=13,
                    scale=1000,
                    offset=None,
                    units="m/s",
                    components=[
                        FitComponentField(
                            name="enhanced_avg_speed",
                            def_num=110,
                            scale=1000,
                            offset=None,
                            units="m/s",
                            accumulate=False,
                            bits=16,
                            bit_offset=0,
                        )
                    ],
                    subfields=None,
                ),
                parent_field=None,
                value=0.0,
                raw_value=0,
                units="m/s",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="max_cadence",
                    type="parse_field_type(record.type)",
                    def_num=18,
                    scale=None,
                    offset=None,
                    units="rpm",
                    components=None,
                    subfields=[
                        FitSubField(
                            name="max_running_cadence",
                            def_num=18,
                            type=FitBaseType(
                                name="uint8",
                                identifier=2,
                                fmt="B",
                                size=1,
                                parse="record.parse",
                            ),
                            scale=None,
                            offset=None,
                            units="strides/min",
                            components=None,
                            ref_fields=[
                                FitReferenceField(
                                    name="sport",
                                    def_num=25,
                                    value="running",
                                    raw_value=1,
                                )
                            ],
                        )
                    ],
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="rpm",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="avg_cadence",
                    type="parse_field_type(record.type)",
                    def_num=17,
                    scale=None,
                    offset=None,
                    units="rpm",
                    components=None,
                    subfields=[
                        FitSubField(
                            name="avg_running_cadence",
                            def_num=17,
                            type=FitBaseType(
                                name="uint8",
                                identifier=2,
                                fmt="B",
                                size=1,
                                parse="record.parse",
                            ),
                            scale=None,
                            offset=None,
                            units="strides/min",
                            components=None,
                            ref_fields=[
                                FitReferenceField(
                                    name="sport",
                                    def_num=25,
                                    value="running",
                                    raw_value=1,
                                )
                            ],
                        )
                    ],
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="rpm",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="max_power",
                    type="parse_field_type(record.type)",
                    def_num=20,
                    scale=None,
                    offset=None,
                    units="watts",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="watts",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="avg_power",
                    type="parse_field_type(record.type)",
                    def_num=19,
                    scale=None,
                    offset=None,
                    units="watts",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="watts",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="max_heart_rate",
                    type="parse_field_type(record.type)",
                    def_num=16,
                    scale=None,
                    offset=None,
                    units="bpm",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="bpm",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="avg_heart_rate",
                    type="parse_field_type(record.type)",
                    def_num=15,
                    scale=None,
                    offset=None,
                    units="bpm",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="bpm",
            ),
            FitFieldData(
                field_def="parse_field_definition(record.field_def)",
                field=FitField(
                    name="total_work",
                    type="parse_field_type(record.type)",
                    def_num=41,
                    scale=None,
                    offset=None,
                    units="J",
                    components=None,
                    subfields=None,
                ),
                parent_field=None,
                value=0,
                raw_value=0,
                units="J",
            ),
        ],
        chunk=None,
    ),
)

(FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type='parse_message_type(record.mesg_type)', global_mesg_num=19, endian='<', field_defs=[FitFieldDefinition(field=FitField(name='timestamp', type='parse_field_type(record.type)', def_num=253, scale=None, offset=None, units='s', components=None, subfields=None), def_num=253, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', size=4, parse='record.parse'), size=4), FitFieldDefinition(field=FitField(name='start_time', type='parse_field_type(record.type)', def_num=2, scale=None, offset=None, units=None, components=None, subfields=None), def_num=2, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', size=4, parse='record.parse'), size=4), FitFieldDefinition(field=FitField(name='total_elapsed_time', type='parse_field_type(record.type)', def_num=7, scale=1000, offset=None, units='s', components=None, subfields=None), def_num=7, base_type=FitBaseType(name='uint32', identifier=134, fmt='I', 

In [75]:
930 / (92)

10.108695652173912

In [5]:
((2 / 3) / (1.25)) * 2

1.0666666666666667